In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
%matplotlib inline
!pip install yfinance
import yfinance as yf
yf.pdr_override()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [ ]:
ticker = "ETH-USD"
nome = ticker
start_date = "2023-05-31"
end_date = "2024-06-11"

# Baixar dados históricos
sol = yf.download(ticker, start=start_date, end=end_date)
rapidMME = sol.Close.ewm(span=12).mean()
lentaMME = sol.Close.ewm(span=26).mean()
MACD = rapidMME - lentaMME
sinal = MACD.ewm(span=9).mean()
sol['MACD'] = MACD
sol['Sinal'] = sinal

[*********************100%%**********************]  1 of 1 completed


In [ ]:
"""plt.figure(figsize=(15,5))
plt.plot(sol.index, MACD, label= nome, color='blue')
plt.plot(sol.index, sinal, label= 'Sinal', color='orange')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.show()"""

"plt.figure(figsize=(15,5))\nplt.plot(sol.index, MACD, label= nome, color='blue')\nplt.plot(sol.index, sinal, label= 'Sinal', color='orange')\nplt.xticks(rotation=90)\nplt.legend(loc='upper right')\nplt.show()"

In [ ]:
"""import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

minimo=min([min(sol['Sinal']),min(sol['MACD'])])
maximo=max([max(sol['Sinal']),max(sol['MACD'])])

fig = make_subplots (vertical_spacing = 0, rows=2, cols=1, row_heights=[4,3])

fig.add_trace(go.Candlestick(x=sol.index, open= sol['Open'], high= sol['High'], low= sol['Low'], close= sol['Close']))

fig.add_trace(go.Scatter(x=sol.index, y= sol['MACD'], name='MACD', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.index, y=sol['Sinal'], name='Sinal', line=dict(color="yellow")), row=2, col=1)


fig.update_layout(  xaxis_rangeslider_visible=False,
                    xaxis=dict(zerolinecolor='black', showticklabels=False),
                    xaxis2=dict(showticklabels=False)
                    )

fig['layout']['yaxis2'].update(range=[minimo, maximo])
fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=False)
fig.show()"""

'import plotly.graph_objects as go\nimport plotly.io as pio\nfrom plotly.subplots import make_subplots\n\nminimo=min([min(sol[\'Sinal\']),min(sol[\'MACD\'])])\nmaximo=max([max(sol[\'Sinal\']),max(sol[\'MACD\'])])\n\nfig = make_subplots (vertical_spacing = 0, rows=2, cols=1, row_heights=[4,3])\n\nfig.add_trace(go.Candlestick(x=sol.index, open= sol[\'Open\'], high= sol[\'High\'], low= sol[\'Low\'], close= sol[\'Close\']))\n\nfig.add_trace(go.Scatter(x=sol.index, y= sol[\'MACD\'], name=\'MACD\', line=dict(color=\'blue\')), row=2, col=1)\nfig.add_trace(go.Scatter(x=sol.index, y=sol[\'Sinal\'], name=\'Sinal\', line=dict(color="yellow")), row=2, col=1)\n\n\nfig.update_layout(  xaxis_rangeslider_visible=False,\n                    xaxis=dict(zerolinecolor=\'black\', showticklabels=False),\n                    xaxis2=dict(showticklabels=False)\n                    )\n\nfig[\'layout\'][\'yaxis2\'].update(range=[minimo, maximo])\nfig.update_xaxes(showline=True, linewidth=1, linecolor="black", mi

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Assume sol is your DataFrame with the necessary data

minimo = min([min(sol['Sinal']), min(sol['MACD'])])
maximo = max([max(sol['Sinal']), max(sol['MACD'])])

# Identify crossover points
crosses_up = (sol['MACD'] > sol['Sinal']) & (sol['MACD'].shift(1) <= sol['Sinal'].shift(1))
crosses_down = (sol['MACD'] < sol['Sinal']) & (sol['MACD'].shift(1) >= sol['Sinal'].shift(1))

# Initialize trading variables
cash = 0
asset = 0
profit = 0
porcentagemTotal = 0

# Track crossover pairs and percentage changes
crossover_pairs = []

# Track profit/loss
for i in range(len(sol)):
    if crosses_up.iloc[i]:
        print(f"Compra efetuada: {sol.index[i]}")
        # Buy $100 worth of asset
        cash -= 100
        asset += 100 / sol['Close'].iloc[i]
        crossover_pairs.append(('up', sol.index[i], sol['Close'].iloc[i]))
    elif crosses_down.iloc[i]:
        print(f"Venda efetuada: {sol.index[i]}")
        # Sell all assets
        if asset > 0:
            cash += asset * sol['Close'].iloc[i]
            asset = 0
            crossover_pairs.append(('down', sol.index[i], sol['Close'].iloc[i]))

# Final profit calculation
profit = cash

fig = make_subplots(vertical_spacing=0, rows=2, cols=1, row_heights=[4, 3])

# Candlestick chart
fig.add_trace(go.Candlestick(x=sol.index, open=sol['Open'], high=sol['High'], low=sol['Low'], close=sol['Close']), row=1, col=1)

# MACD and Sinal lines
fig.add_trace(go.Scatter(x=sol.index, y=sol['MACD'], name='MACD', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.index, y=sol['Sinal'], name='Sinal', line=dict(color='yellow')), row=2, col=1)

# Green markers for upward crossovers
fig.add_trace(go.Scatter(
    x=sol.index[crosses_up],
    y=sol['MACD'][crosses_up],
    mode='markers',
    marker=dict(color='green', size=10),
    name='Cross Up'
), row=2, col=1)

# Red markers for downward crossovers
fig.add_trace(go.Scatter(
    x=sol.index[crosses_down],
    y=sol['MACD'][crosses_down],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Cross Down'
), row=2, col=1)

# Add gray lines and percentage change annotations
for i in range(1, len(crossover_pairs), 2):
    if crossover_pairs[i-1][0] == 'up' and crossover_pairs[i][0] == 'down':
        x0, y0 = crossover_pairs[i-1][1], crossover_pairs[i-1][2]
        x1, y1 = crossover_pairs[i][1], crossover_pairs[i][2]
        percentage_change = ((y1 - y0) / y0) * 100
        # Midpoint for annotation
        midpoint_x = x0 + (x1 - x0) / 2
        midpoint_y = sol['MACD'][x0] + (sol['MACD'][x1] - sol['MACD'][x0]) / 2
        fig.add_trace(go.Scatter(
            x=[x0, x1],
            y=[sol['MACD'][x0], sol['MACD'][x1]],
            mode='lines+text',
            line=dict(color='gray', width=1, dash='dash'),
            text=[None, f'   {percentage_change:.2f}%'],
            textposition='middle right',
            showlegend=False
        ), row=2, col=1)
        porcentagemTotal = (porcentagemTotal + percentage_change)

# Add profit/loss to the legend
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='white'),
    name=f'Porcentagem Total: {porcentagemTotal:.2f}%'

))

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(zerolinecolor='black', showticklabels=False),
    xaxis2=dict(showticklabels=False),
    legend=dict(yanchor="top", y=1.1, xanchor="left", x=0.01),
    height=900,  # Increase the height
    width=1750,  # Increase the width
    margin=dict(l=20, r=20, t=20, b=20)  # Reduce margins to use more space
)

fig['layout']['yaxis2'].update(range=[minimo, maximo])
fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=False)
fig.show()

Venda efetuada: 2023-06-01 00:00:00
Compra efetuada: 2023-06-02 00:00:00
Venda efetuada: 2023-06-05 00:00:00
Compra efetuada: 2023-06-20 00:00:00
Venda efetuada: 2023-07-07 00:00:00
Compra efetuada: 2023-07-13 00:00:00
Venda efetuada: 2023-07-18 00:00:00
Compra efetuada: 2023-08-10 00:00:00
Venda efetuada: 2023-08-15 00:00:00
Compra efetuada: 2023-08-29 00:00:00
Venda efetuada: 2023-09-11 00:00:00
Compra efetuada: 2023-09-13 00:00:00
Venda efetuada: 2023-10-09 00:00:00
Compra efetuada: 2023-10-20 00:00:00
Venda efetuada: 2023-11-16 00:00:00
Compra efetuada: 2023-12-03 00:00:00
Venda efetuada: 2023-12-12 00:00:00
Compra efetuada: 2023-12-28 00:00:00
Venda efetuada: 2023-12-29 00:00:00
Compra efetuada: 2024-01-02 00:00:00
Venda efetuada: 2024-01-03 00:00:00
Compra efetuada: 2024-01-10 00:00:00
Venda efetuada: 2024-01-20 00:00:00
Compra efetuada: 2024-02-06 00:00:00
Venda efetuada: 2024-03-15 00:00:00
Compra efetuada: 2024-04-08 00:00:00
Venda efetuada: 2024-04-12 00:00:00
Compra efetuada

In [ ]:
sol.index[0]

Timestamp('2024-01-01 00:00:00')

In [ ]:
sol['Close'].index[145]


Timestamp('2024-05-26 00:00:00')